In [4]:
from hatchet import *
import pandas as pd
import json

In [40]:
path = "/home/vidi/Work/llnl/CallFlow/data/kripke/"
datasets = ['impi', 'mvapich2']

graph_files = {}
df_files = {}
graph_file = 'graph.json'
for dataset in datasets:
    graph_files[dataset] = path + dataset + '/' + graph_file
    
df_file = 'df.csv'
for dataset in datasets:
    df_files[dataset] = path + dataset + '/' + df_file
    
gfs = {}
for idx, dataset in enumerate(graph_files):
    with open(graph_files[dataset], 'r') as graphFile:
        data = json.load(graphFile)
        gf = GraphFrame()
        gf.from_literal(data)
        gfs[dataset] = gf
        gfs[dataset].df = pd.read_csv(df_files[dataset])
print(gfs)

{'impi': <hatchet.graphframe.GraphFrame object at 0x7f28f38f6e10>, 'mvapich2': <hatchet.graphframe.GraphFrame object at 0x7f28f3874198>}


In [98]:
graph = gfs['impi'].graph
df = gfs['impi'].df

In [23]:
def dfs(graph, limit):
    
    def dfs_recurse(root):
        for node in root.children:
#             if(level < limit):
            print("Node : ", node)
            dfs_recurse(node)
        
    for root in graph.roots:
        dfs_recurse(root)

In [100]:
class GroupNode:
    def __init__(self, module):
        self.module = module
        
    def add_properties(self, nid, callpath_tuple, parent):
        self.n_propertiesadd
        self.module = module
        self.callpath = callpath_tuple
        
        self.parents = []
        if parent is not None:
            self.add_parent(parent)
        self.children = []
        self.entry_funcs = []
        self.inner_nodes = []

    def add_parent(self, node):
        assert isinstance(node, groupNode)
        self.parents.append(node)

    def add_child(self, node):
        assert isinstance(node, groupNode)
        self.children.append(node)

    def add_entry_funcs(self, node):
        assert isinstance(node, Node)
        self.entry_funcs.append(node)

    def __str__(self):
        """ Returns a string representation of the node.
        """
        return 'nid: {0}, number_of_parents: {1}, number_of_children: {2}, entry_funcs: {3}'.format(self.nid, len(self.parents), len(self.children), len(self.entry_funcs))


In [93]:
class groupGraph:
    """ A group node in the call graph.
    """
    def __init__(self):
        self.nodes = {}
        self.inner_nodes = []

    def add_module_name(self, name):
        self.modules.append(name)
        
    def add_group_node(self, node, name):
        assert isinstance(node, groupNode)
        self.nodes[name] = node

In [72]:
def create_group_path(path):
    group_path = []
    temp = None
    for i, elem in enumerate(path):
        elem_df = df.loc[df.name == elem]
        if(elem_df.empty):
            grouping = 'Unkno'
        else:
            grouping = elem_df['module'].unique()[0]
        if temp == None or grouping != temp:
            group_path.append(grouping)
            temp = grouping
    return tuple(group_path)

In [82]:
# Assign a "Module" name to a given hatchet node. 
def group_by_name(df, node):
    # start = time.time()
    df = df.loc[df.name == node.callpath[-1]]
    unique_modules = df['module'].unique()
    if(len(unique_modules) == 1):
        return unique_modules[0]
    elif(len(unique_modules) == 0): 
        return 'Unkno'
    else:
        print('Error! Multiple modules for a node.')
        return None

In [103]:
modules = df['module'].unique()

g_graph = groupGraph()
for idx, module in enumerate(modules):
    module_groupNode = groupNode(module)
    g_graph.add_group_node(module_groupNode, module)

print(g_graph.nodes)

TypeError: 'groupNode' object is not callable

In [96]:
for root in graph.roots:
    node_gen = root.traverse()
    new_name = group_by_name(df, root)
    new_callpath = create_group_path(root.callpath)
    hatchet_hash = df.loc[df.name == root.callpath[-1]]['node'].unique()
    
    count = 0
    
    groupNode = g_graph.nodes[new_name]
    print(groupNode)
    new_group_node = groupNode.add(count, new_callpath, None)
    new_group_node.add_entry_funcs(root)
    
    try: 
        while root.callpath != None:
            root = next(node_gen)
            new_name = group_by_name(df, root)
            new_callpath = create_group_path(root.callpath)
            hatchet_hash = df.loc[df.name == root.callpath[-1]]['node'].unique()
            
            new_group_node = groupNode(count, new_name, new_callpath, None)
            new_group_node.add_entry_funcs(root)
            
            print(root, new_name)
    except StopIteration:
        pass
    finally:
        del root

TypeError: 'groupNode' object is not callable